# Dummy CLIP finetuning example

In this notebook we test a basic code to finetune CLIP on a given dataset of (images, texts).
As dataset, we provide 3 pictures from the internet, depicting a cat, a dog and a turtle. We finetune the model on a dummy dataset that maps these images to their respective labels translated in Italian. 

This code trains the model using Tensorflow API. Similar examples can be also implemented using Pytorch or Hugginface high-level API.

In [1]:
import tensorflow as tf
from PIL import Image
import requests
from transformers import CLIPProcessor, TFCLIPModel

model = TFCLIPModel.from_pretrained("openai/clip-vit-base-patch32") #load CLIP as a TF model
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url1 = "http://images.cocodataset.org/val2017/000000039769.jpg"
image1 = Image.open(requests.get(url1, stream=True).raw)

url2 = "http://www.veterinarimatera.it/wp-content/uploads/2021/03/cane-1.jpg"
image2 = Image.open(requests.get(url2, stream=True).raw)

url3 = "https://www.cure-naturali.it/.imaging/default/dam/cure-naturali/enciclopedia-naturale/animali/tartaruga-di-terra.jpg/jcr:content.jpg"
image3 = Image.open(requests.get(url3, stream=True).raw)

inputs = processor(
    text=["cat", "dog", "turtle"], images=[image1, image2,image3], return_tensors="tf", padding=True
)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = tf.nn.softmax(logits_per_image, axis=1)
print(probs)

All model checkpoint layers were used when initializing TFCLIPModel.

All the layers of TFCLIPModel were initialized from the model checkpoint at openai/clip-vit-base-patch32.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCLIPModel for predictions without further training.


tf.Tensor(
[[9.7333241e-01 6.5846965e-03 2.0082889e-02]
 [6.9013826e-04 9.9824983e-01 1.0600548e-03]
 [1.5486321e-05 1.0035410e-04 9.9988413e-01]], shape=(3, 3), dtype=float32)


The matrix above represents the similarity of each photo (on the column axis) to each of the 3 proposed labels on the row axis. We can see how the model is correctly working; the max value in the first column, representing the similarities of the "cat" photo to each of the label, is in the first row, associated to the "cat" label. We can find the same pattern in the other columns as well.

Now we test the model on the respective Italian translations. 

In [2]:
dummy_training_sample = processor(
    text=["gatto", "cane", "tartaruga"], images=[image1, image2,image3], return_tensors="tf", padding=True
)


In [3]:
outputs = model(**dummy_training_sample)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = tf.nn.softmax(logits_per_image, axis=1)
print(probs)

tf.Tensor(
[[0.8833075  0.05306344 0.06362902]
 [0.00273138 0.9601945  0.03707418]
 [0.00401178 0.01271232 0.9832759 ]], shape=(3, 3), dtype=float32)


The predictions are still surprisingly correct (some Italian-labeled pictures slipped in into the original CLIP dataset?), but we can see how the similarity scores are slightly lower.

Our training code will "overfit" the model on these 3 samples. The below code is written as a custom training loop using TF API to get a better flow control, nonetheless Keras API  *(model.compile(...), model.fit(...)...)* will work as well. 
The raw images and texts must be passed through the CLIP processor object, in order to tokenize the texts and normalize the images. The processed samples are then passed to the model.

The training pseudocode (from the CLIP paper) is defined as:

```
    # text_encoder - CBOW or Text Transformer
    # I[n, h, w, c] - minibatch of aligned images
    # T[n, l] - minibatch of aligned texts
    # W_i[d_i, d_e] - learned proj of image to embed
    # W_t[d_t, d_e] - learned proj of text to embed
    # t - learned temperature parameter
    # extract feature representations of each modality
    I_f = image_encoder(I) #[n, d_i]
    T_f = text_encoder(T) #[n, d_t]
    # joint multimodal embedding [n, d_e]
    I_e = l2_normalize(np.dot(I_f, W_i), axis=1)
    T_e = l2_normalize(np.dot(T_f, W_t), axis=1)
    # scaled pairwise cosine similarities [n, n]
    logits = np.dot(I_e, T_e.T) * np.exp(t)
    # symmetric loss function
    labels = np.arange(n)
    loss_i = cross_entropy_loss(logits, labels, axis=0)
    loss_t = cross_entropy_loss(logits, labels, axis=1)
    loss = (loss_i + loss_t)/2
```

`TFCLIP_Model` automatically provides the image and text logits, so we just need to grab them and apply the losses. In order to create the ground truth, we needed to replace the `arange` function with the `eye` function (identity square matrix) to match the logits tensor shapes. Notice how the `eye` function is equivalent to one-hot encoding the labels created with the `arange` function.

Our objective is not to get a working model on Italian labels, but to simply show how the model can be finetuned on more meaningful tasks.

In [4]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

img_loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
txt_loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

for epoch in range(30):
    #Add code to iterate over different batches. This code trains the model over a single batch.
    with tf.GradientTape() as tape:
        batch_size = 3 #You should write code to infer the correct batch size from the training sample
        out = model(**dummy_training_sample, training=True)

        ground_truth = tf.eye(batch_size, dtype=tf.float32, name='gt')
        total_loss = (img_loss_fn(ground_truth, out.logits_per_image) + txt_loss_fn(ground_truth, out.logits_per_text))/2

    grads = tape.gradient(total_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    print(total_loss)
    
    #Add callbacks, metrics, evaluation or whatever you want

tf.Tensor(0.11198809, shape=(), dtype=float32)
tf.Tensor(4.098654, shape=(), dtype=float32)
tf.Tensor(0.025565138, shape=(), dtype=float32)
tf.Tensor(0.96216196, shape=(), dtype=float32)
tf.Tensor(3.712772, shape=(), dtype=float32)
tf.Tensor(0.929594, shape=(), dtype=float32)
tf.Tensor(1.0677291, shape=(), dtype=float32)
tf.Tensor(0.36412066, shape=(), dtype=float32)
tf.Tensor(0.17473613, shape=(), dtype=float32)
tf.Tensor(0.08043531, shape=(), dtype=float32)
tf.Tensor(0.034443527, shape=(), dtype=float32)
tf.Tensor(0.06659063, shape=(), dtype=float32)
tf.Tensor(0.026157139, shape=(), dtype=float32)
tf.Tensor(0.0045526205, shape=(), dtype=float32)
tf.Tensor(0.0011914971, shape=(), dtype=float32)
tf.Tensor(0.00069613993, shape=(), dtype=float32)
tf.Tensor(0.0006224165, shape=(), dtype=float32)
tf.Tensor(0.0006280744, shape=(), dtype=float32)
tf.Tensor(0.00064741797, shape=(), dtype=float32)
tf.Tensor(0.0006604326, shape=(), dtype=float32)
tf.Tensor(0.00065866683, shape=(), dtype=float32

In [5]:
outputs = model(**dummy_training_sample)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = tf.nn.softmax(logits_per_image, axis=1)
print(probs)

tf.Tensor(
[[9.9948764e-01 5.1222561e-04 1.2926184e-07]
 [1.2395257e-04 9.9987566e-01 3.8258275e-07]
 [5.6019152e-04 5.3928702e-08 9.9943978e-01]], shape=(3, 3), dtype=float32)


We can see how the new similarities are very high on the training samples, as expected.

After training and evaluating the model, we can save the model using the `save_pretrained` HuggingFace method. The model might be saved using the TF API as well (`model.save(...)`, `tf.keras.models.load_model(...)`), but then we must use TF API to invoke the model, but we need to know the TF expected input format, which is not clear at a first glance, so using the HuggingFace API comes handier. We can also exploit the HuggingFace processor, which is responsible to tokenize the texts in a proper way.

In [6]:
model.save_pretrained('../data/test_clip_train')

In [7]:
model_loaded = TFCLIPModel.from_pretrained('../data/test_clip_train',local_files_only=True)

All model checkpoint layers were used when initializing TFCLIPModel.

All the layers of TFCLIPModel were initialized from the model checkpoint at ../data/test_clip_train.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCLIPModel for predictions without further training.


In [8]:
outputs = model_loaded(**dummy_training_sample)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = tf.nn.softmax(logits_per_image, axis=1)
print(probs)

tf.Tensor(
[[9.9948764e-01 5.1222561e-04 1.2926184e-07]
 [1.2395257e-04 9.9987566e-01 3.8258275e-07]
 [5.6019152e-04 5.3928702e-08 9.9943978e-01]], shape=(3, 3), dtype=float32)
